In [ ]:
path = ""
url = ""
id = -1

In [ ]:
from astropy.io import fits
import numpy as np
from init_val_generator import InitValGenerator

image_data = fits.getdata(path)

index = 0
while image_data.shape[index] == 1:
    index = index + 1
height = image_data.shape[index]
width = image_data.shape[index + 1]

image_data = np.reshape(image_data, width * height)

guesser = InitValGenerator("2-fwhm-estimate", "3-sigma")
estimates = guesser.estimate(image_data, width, height, 3)
print(estimates)

In [ ]:
from carta.session import Session

session = Session.interact(url, id)

In [ ]:
img1 = session.open_image(path)
img1.set_zoom_level(10)

In [ ]:
for param in estimates:
    amp, center_x, center_y, fwhm_x, fwhm_y, pa = param
    region_id = img1.call_action("regionSet.addRegionAsync", 4, [{"x": center_x, "y": center_y}, {"x": fwhm_x / 2, "y": fwhm_y / 2}], pa, "test", return_path="regionId")
    # img1.regions.add_ellipse([center_x, center_y], [fwhm_x / 2, fwhm_y / 2], rotation=int(pa))

In [ ]:
from IPython.display import Image

picture = Image(data=session.rendered_view_data("white"),width=800)
display(picture)

In [ ]:
session.call_action("dialogStore.showDialog", "fitting-dialog")

In [ ]:
session.call_action("imageFittingStore.clearComponents")
session.call_action("imageFittingStore.setComponents", len(estimates))

for i in range(len(estimates)):
    amp, center_x, center_y, fwhm_x, fwhm_y, pa = estimates[i]
    session.call_action("imageFittingStore.components[{}].setCenterX".format(i), center_x)
    session.call_action("imageFittingStore.components[{}].setCenterY".format(i), center_y)
    session.call_action("imageFittingStore.components[{}].setAmplitude".format(i), amp)
    session.call_action("imageFittingStore.components[{}].setFwhmX".format(i), fwhm_x)
    session.call_action("imageFittingStore.components[{}].setFwhmY".format(i), fwhm_y)
    session.call_action("imageFittingStore.components[{}].setPa".format(i), pa)

In [ ]:
session.call_action("imageFittingStore.fitImage")

In [ ]:
session.call_action("imageFittingStore.createRegions")

In [ ]:
picture = Image(data=session.rendered_view_data("white"),width=1200)
display(picture)